In [1]:
import pickle
import itertools
import numpy as np
from scipy import spatial
from scipy.stats import norm
import nltk.data
from nltk import pos_tag
from nltk.corpus import wordnet as wn
from nltk.corpus import reuters
from nltk. corpus import gutenberg
from nltk.corpus import brown
from nltk.tokenize import sent_tokenize
from gensim.models import KeyedVectors
from keras.layers import Input, Dense, Lambda, Layer
from keras.callbacks import ModelCheckpoint
from keras.models import Model
from keras import backend as K
from keras import metrics

C:\Users\Vinod\Anaconda2\envs\tensorflow\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
C:\Users\Vinod\Anaconda2\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Preprocessing Text

The preprocessing code is data specific.  
  
It is an example of how one can use a pre-trained word2vec to embed sentences into a vector space.

In [2]:
def split_into_sent (text):
    strg = ''
    for word in text:
        strg += word
        strg += ' '
    strg_cleaned = strg.lower()
    for x in ['\xd5d','\n','"',"!", '#','$','%','&','(',')','*','+',',','-','/',':',';','<','=','>','?','@','[','^',']','_','`','{','|','}','~','\t']:
        strg_cleaned = strg_cleaned.replace(x, '')
    sentences = sent_tokenize(strg_cleaned)
    return sentences

In [3]:
def vectorize_sentences(sentences):
    vectorized = []
    for sentence in sentences:
        byword = sentence.split()
        concat_vector = []
        for word in byword:
            try:
                concat_vector.append(w2v[word])
            except:
                pass
        vectorized.append(concat_vector)
    return vectorized

Preprocessing text from a variety of different sources.

In [4]:
import fasttext
data_concat = []
all_text=[]
txt_indx=0


#text = split_into_sent(brown.words())


    
model = fasttext.skipgram("APT_sanitized.txt", 'apt_sanitized',dim=100, min_count=0)

In [4]:
w2v = KeyedVectors.load_word2vec_format('apt_vectors.vec')
data_concat = []


with open('APT_sanitized.txt',"r",encoding='utf-8') as f:
    text=f.readlines()
vect = vectorize_sentences(text)
data = [x for x in vect  if len(x) == 10]
for x in data:
    data_concat.append(list(itertools.chain.from_iterable(x)))

It's important to shuffle the text vectors before splitting them into test and train samples.   
  
This is done to avoid clumping text with similar context and style in the dataset because it can confuse the neural network during training.

In [17]:
vect[0][0]

array([ 0.00227 , -0.002735,  0.004701, -0.004814, -0.001183, -0.002806,
        0.000746,  0.004251, -0.004267, -0.000771, -0.011192, -0.001916,
        0.003505,  0.003675,  0.006862, -0.00515 , -0.005831, -0.00332 ,
        0.009895, -0.004621,  0.005789, -0.001071,  0.002866, -0.006613,
       -0.000891, -0.006907,  0.007527,  0.005952, -0.00475 , -0.001488,
       -0.005064,  0.010672,  0.005715, -0.001045, -0.009741,  0.001058,
       -0.002325, -0.001972,  0.000217, -0.000577,  0.00537 ,  0.000388,
        0.001798,  0.003835, -0.001138, -0.001426,  0.003441,  0.007415,
       -0.000421, -0.007193, -0.002982, -0.000531, -0.003634, -0.002289,
       -0.000626,  0.004089, -0.001688,  0.002588,  0.004141,  0.010842,
        0.00107 ,  0.006035, -0.005503,  0.016965, -0.001544,  0.005332,
        0.004329, -0.002427,  0.009539,  0.001806,  0.002386,  0.001555,
       -0.000354,  0.001823, -0.003668,  0.006647,  0.000238,  0.006995,
       -0.002169,  0.000718, -0.000288, -0.001508, 

In [15]:
data_concat[0]

[0.00227,
 -0.002735,
 0.004701,
 -0.004814,
 -0.001183,
 -0.002806,
 0.000746,
 0.004251,
 -0.004267,
 -0.000771,
 -0.011192,
 -0.001916,
 0.003505,
 0.003675,
 0.006862,
 -0.00515,
 -0.005831,
 -0.00332,
 0.009895,
 -0.004621,
 0.005789,
 -0.001071,
 0.002866,
 -0.006613,
 -0.000891,
 -0.006907,
 0.007527,
 0.005952,
 -0.00475,
 -0.001488,
 -0.005064,
 0.010672,
 0.005715,
 -0.001045,
 -0.009741,
 0.001058,
 -0.002325,
 -0.001972,
 0.000217,
 -0.000577,
 0.00537,
 0.000388,
 0.001798,
 0.003835,
 -0.001138,
 -0.001426,
 0.003441,
 0.007415,
 -0.000421,
 -0.007193,
 -0.002982,
 -0.000531,
 -0.003634,
 -0.002289,
 -0.000626,
 0.004089,
 -0.001688,
 0.002588,
 0.004141,
 0.010842,
 0.00107,
 0.006035,
 -0.005503,
 0.016965,
 -0.001544,
 0.005332,
 0.004329,
 -0.002427,
 0.009539,
 0.001806,
 0.002386,
 0.001555,
 -0.000354,
 0.001823,
 -0.003668,
 0.006647,
 0.000238,
 0.006995,
 -0.002169,
 0.000718,
 -0.000288,
 -0.001508,
 0.00395,
 -0.000166,
 -0.009835,
 0.008755,
 -0.001973,
 -0.0

In [5]:
data_array = np.array(data_concat)
np.random.shuffle(data_array)

train = data_array[:60000]
test = data_array[60000:80000]

In [9]:
import numpy as np

len(train[10])

1000

# Variational Autoencoder

In [6]:
import keras
import h5py

batch_size = 50
original_dim = 1000
latent_dim = 100
intermediate_dim = 1200
epochs = 100
epsilon_std = 1.0

x = Input(batch_shape=(batch_size, original_dim))
h = Dense(intermediate_dim, activation='relu')(x)
z_mean = Dense(latent_dim)(h)
z_log_var = Dense(latent_dim)(h)

def sampling(args):
    z_mean, z_log_var = args
    epsilon = K.random_normal(shape=(batch_size, latent_dim), mean=0.,
                              stddev=epsilon_std)
    return z_mean + K.exp(z_log_var / 2) * epsilon

# note that "output_shape" isn't necessary with the TensorFlow backend
z = Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_var])

# we instantiate these layers separately so as to reuse them later
decoder_h = Dense(intermediate_dim, activation='relu')
decoder_mean = Dense(original_dim, activation='sigmoid')
h_decoded = decoder_h(z)
x_decoded_mean = decoder_mean(h_decoded)

# placeholder loss
def zero_loss(y_true, y_pred):
    return K.zeros_like(y_pred)

# Custom loss layer
class CustomVariationalLayer(Layer):
    def __init__(self, **kwargs):
        self.is_placeholder = True
        super(CustomVariationalLayer, self).__init__(**kwargs)

    def vae_loss(self, x, x_decoded_mean):
        xent_loss = original_dim * metrics.binary_crossentropy(x, x_decoded_mean)
        kl_loss = - 0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
        return K.mean(xent_loss + kl_loss)

    def call(self, inputs):
        x = inputs[0]
        x_decoded_mean = inputs[1]
        loss = self.vae_loss(x, x_decoded_mean)
        self.add_loss(loss, inputs=inputs)
        # we don't use this output, but it has to have the correct shape:
        return K.ones_like(x)

loss_layer = CustomVariationalLayer()([x, x_decoded_mean])
vae = Model(x, [loss_layer])
vae.compile(optimizer='rmsprop', loss=[zero_loss])

#checkpoint
cp = [keras.callbacks.ModelCheckpoint(filepath="model.h5", verbose=1, save_best_only=True)]

#train
vae.fit(train, train,
        shuffle=True,
        epochs=epochs,
        batch_size=batch_size,
        validation_data=(test, test), callbacks=cp)

# build a model to project inputs on the latent space
encoder = Model(x, z_mean)

# build a generator that can sample from the learned distribution
decoder_input = Input(shape=(latent_dim,))
_h_decoded = decoder_h(decoder_input)
_x_decoded_mean = decoder_mean(_h_decoded)
generator = Model(decoder_input, _x_decoded_mean)

Train on 60000 samples, validate on 20000 samples
Epoch 1/100
59850/60000 [============================>.] - ETA: 0s - loss: -4189.2157
Epoch 00001: val_loss improved from inf to -4479.34883, saving model to model.h5
60000/60000 [==============================] - 17s 288us/step - loss: -4190.0944 - val_loss: -4479.3488
Epoch 2/100
59750/60000 [============================>.] - ETA: 0s - loss: -4546.2459
Epoch 00002: val_loss improved from -4479.34883 to -4626.42594, saving model to model.h5
60000/60000 [==============================] - 10s 168us/step - loss: -4546.7003 - val_loss: -4626.4259
Epoch 3/100
59950/60000 [============================>.] - ETA: 0s - loss: -4653.6747
Epoch 00003: val_loss improved from -4626.42594 to -4713.03336, saving model to model.h5
60000/60000 [==============================] - 10s 172us/step - loss: -4653.4839 - val_loss: -4713.0334
Epoch 4/100
59950/60000 [============================>.] - ETA: 0s - loss: -4712.7016
Epoch 00004: val_loss improved from

Epoch 32/100
59950/60000 [============================>.] - ETA: 0s - loss: -4926.7962
Epoch 00032: val_loss did not improve
60000/60000 [==============================] - 11s 183us/step - loss: -4926.4869 - val_loss: -4929.6345
Epoch 33/100
59850/60000 [============================>.] - ETA: 0s - loss: -4928.8307
Epoch 00033: val_loss did not improve
60000/60000 [==============================] - 11s 179us/step - loss: -4928.7720 - val_loss: -4936.9719
Epoch 34/100
59750/60000 [============================>.] - ETA: 0s - loss: -4930.2240
Epoch 00034: val_loss improved from -4938.74937 to -4944.85326, saving model to model.h5
60000/60000 [==============================] - 11s 180us/step - loss: -4929.9757 - val_loss: -4944.8533
Epoch 35/100
59850/60000 [============================>.] - ETA: 0s - loss: -4931.2191
Epoch 00035: val_loss did not improve
60000/60000 [==============================] - 10s 168us/step - loss: -4931.1220 - val_loss: -4941.7179
Epoch 36/100
59600/60000 [=======

59700/60000 [============================>.] - ETA: 0s - loss: -4957.8440
Epoch 00066: val_loss did not improve
60000/60000 [==============================] - 11s 175us/step - loss: -4959.0370 - val_loss: -4962.9457
Epoch 67/100
59650/60000 [============================>.] - ETA: 0s - loss: -4960.2068
Epoch 00067: val_loss did not improve
60000/60000 [==============================] - 11s 176us/step - loss: -4959.5509 - val_loss: -4963.1348
Epoch 68/100
59800/60000 [============================>.] - ETA: 0s - loss: -4959.9270
Epoch 00068: val_loss improved from -4966.21839 to -4970.41985, saving model to model.h5
60000/60000 [==============================] - 11s 178us/step - loss: -4960.3153 - val_loss: -4970.4199
Epoch 69/100
59600/60000 [============================>.] - ETA: 0s - loss: -4961.2962
Epoch 00069: val_loss improved from -4970.41985 to -4974.52378, saving model to model.h5
60000/60000 [==============================] - 11s 180us/step - loss: -4961.2603 - val_loss: -4974.

59950/60000 [============================>.] - ETA: 0s - loss: -4971.4444
Epoch 00100: val_loss did not improve
60000/60000 [==============================] - 11s 175us/step - loss: -4971.3923 - val_loss: -4976.7031


# Generating Text From Latent Space

In [7]:
# some matrix magic
def sent_parse(sentence, mat_shape):
    data_concat = []
    word_vecs = vectorize_sentences(sentence)
    for x in word_vecs:
        data_concat.append(list(itertools.chain.from_iterable(x)))
    zero_matr = np.zeros(mat_shape)
    zero_matr[0] = np.array(data_concat)
    return zero_matr

In [8]:
# input: original dimension sentence vector
# output: text
def print_sentence_with_w2v(sent_vect):
    word_sent = ''
    tocut = sent_vect
    for i in range (int(len(sent_vect)/100)):
        word_sent += w2v.most_similar(positive=[tocut[:100]], topn=1)[0][0]
        word_sent += ' '
        tocut = tocut[100:]
    print(word_sent)

In [9]:
# input: encoded sentence vector
# output: encoded sentence vector in dataset with highest cosine similarity
def find_similar_encoding(sent_vect):
    all_cosine = []
    for sent in sent_encoded:
        result = 1 - spatial.distance.cosine(sent_vect, sent)
        all_cosine.append(result)
    data_array = np.array(all_cosine)
    maximum = data_array.argsort()[-3:][::-1][1]
    new_vec = sent_encoded[maximum]
    return new_vec

In [10]:
# input: two points, integer n
# output: n equidistant points on the line between the input points (inclusive)
def shortest_homology(point_one, point_two, num):
    dist_vec = point_two - point_one
    sample = np.linspace(0, 1, num, endpoint = True)
    hom_sample = []
    for s in sample:
        hom_sample.append(point_one + s * dist_vec)
    return hom_sample

In [11]:
# input: two written sentences, VAE batch-size, dimension of VAE input
# output: the function embeds the sentences in latent-space, and then prints their generated text representations
# along with the text representations of several points in between them
def sent_2_sent(sent1,sent2, batch, dim):
    a = sent_parse([sent1], (batch,dim))
    b = sent_parse([sent2], (batch,dim))
    encode_a = encoder.predict(a, batch_size = batch)
    encode_b = encoder.predict(b, batch_size = batch)
    test_hom = shortest_homology(encode_a[0], encode_b[0], 5)
    
    for point in test_hom:
        p = generator.predict(np.array([point]))[0]
        print_sentence_with_w2v(p)

Printing sentences from the training set and comparing them with the original will test whether the custom print function works properly.

In [15]:

print_sentence_with_w2v(train[15])
print_sentence_with_w2v(train[20])
sent_2_sent('already legitimately used in an Internet Explorer plugin open source','depending on Windows version . Returns failure or success with',batch=50,dim=1000)

copy cached user credentials from web browsers and chat clients 
actors left phpinfo.php script on the webserver running on port 
limited webfacing used fictitious this infected workflow 104.238.184.252 Cisco3045EK9 fictitious 
revealed webfacing used fictitious the webfacing Bit9signed provided Cisco3045EK9 fictitious 
limited fictitious Whether fictitious available webfacing nonsensitive Webbased Cisco3045EK9 Webbased 
provided on Launches main analysis webfacing nonsensitive Webbased Cisco3045EK9 over 
obtained on Launches following . cyberterrorists •httpwww.lemonde.frpixelsarticle20150409lessitesdetv5mondedetournesparungroupe Webbased stolen over 


The encoder takes the training set of sentence vectors (concatenanted word vectors) and embeds them into a lower dimensional vector space.

In [17]:
sent_encoded = encoder.predict(np.array(train), batch_size = 50)

The decoder takes the list of latent dimensional encodings from above and turns them back into vectors of their original dimension.

In [18]:
sent_decoded = generator.predict(sent_encoded)
print(sent_decoded.shape)

(60000, 1000)


The encoder trained above embeds sentences (concatenated word vetors) into a lower dimensional space. The code below takes two of these lower dimensional sentence representations and finds five points between them. It then uses the trained decoder to project these five points into the higher, original, dimensional space. Finally, it reveals the text represented by the five generated sentence vectors by taking each word vector concatenated inside and finding the text associated with it in the word2vec used during preprocessing.

In [37]:


test_hom = shortest_homology(sent_encoded[105], sent_encoded[107], 10)
for point in test_hom:
    p = generator.predict(np.array([point]))[0]
    print_sentence_with_w2v(p)

devised 4e84b1448cf96fabe88c623b222057c4 cyberterrorists families relayed crossplatform different methods Classic 0BCEh—webcam 
webfacing 4e84b1448cf96fabe88c623b222057c4 pwNative.exe families relayed crossplatform different methods Classic 104.238.184.252 
webfacing 4e84b1448cf96fabe88c623b222057c4 month.30 families different further different different Classic 104.238.184.252 
webfacing Battery month.30 different different traits different different Classic •httpwww.lemonde.frpixelsarticle20150409lessitesdetv5mondedetournesparungroupe 
webfacing update.msntoole.com Sakula different different common different different wiper webfacing 
compromised webfacing primary different different common different different 104.238.184.252 webfacing 
infected Russiabased . relayed different common filtered different 104.238.184.252 webfacing 
infected Russiabased . relayed different common filtered relayed webfacing webfacing 
infected government . relayed different common filtered relayed webfaci

The code below does the same thing, with one important difference. After sampling equidistant points in the latent space between two sentence embeddings, it finds the embeddings from our encoded dataset those points are most similar to. It then prints the text associated with those vectors.
  
This allows us to explore how the Variational Autoencoder clusters our dataset of sentences in latent space. It lets us investigate whether sentences with similar concepts or grammatical styles are represented in similar areas of the lower dimensional space.

In [38]:
test_hom = shortest_homology(sent_encoded[2], sent_encoded[1500], 20)
for point in test_hom:
    p = generator.predict(np.array([find_similar_encoding(point)]))[0]
    print_sentence_with_w2v(p)

precise threat targeted fictitious the familyBackdoor.Winnti devised fictitious new signiﬁcant 
precise threat targeted fictitious the familyBackdoor.Winnti devised fictitious new signiﬁcant 
precise threat targeted fictitious the familyBackdoor.Winnti devised fictitious new signiﬁcant 
infected a infected fictitious the webfacing Bit9signed of the Bit9signed 
infected a infected fictitious the webfacing Bit9signed of the Bit9signed 
memorystorage a new memorystorage its Bit9signed webfacing Bit9signed the different 
memorystorage a new memorystorage its Bit9signed webfacing Bit9signed the different 
memorystorage a new memorystorage its Bit9signed webfacing Bit9signed the different 
memorystorage a new memorystorage its Bit9signed webfacing Bit9signed the different 


KeyboardInterrupt: 